<a href="https://colab.research.google.com/github/KOdin2/Machine-learning/blob/main/Random_forest_Binary_classifier_assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import io
import pandas as pd
import sklearn
import csv


#####
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import files
uploaded = files.upload()

Saving SSteel_25-75WF_dataset.csv to SSteel_25-75WF_dataset.csv


Best measurements to use for classification are:

precision,
recall,
accuracy,
f1,
roc_auc


In [3]:
#@title Functions

def grid_search_RF_function(x_train, y_train, feature_combinations, varible_combinations, number_of_cv_folds, best_results, x_train_df):

  Algorithm = "Random Forest"
  forest_reg = RandomForestClassifier(random_state=0)

  #Set up parameter grid
  param_grid = [
      {'bootstrap': [False], 'n_estimators': varible_combinations, 'max_features': feature_combinations}
  ]

  #scoring_type can be used to choose what method to compare the models on
  scoring_type = 'f1'

  print("Number of fold: " + str(number_of_cv_folds))
  
  grid_search = GridSearchCV(estimator= forest_reg, param_grid= param_grid,  cv= number_of_cv_folds , scoring=scoring_type,  return_train_score=True, n_jobs=-1)

  grid_search.fit(X= x_train, y= y_train)

  best_paramater = grid_search.best_params_

  print("[INFO]Best paramter: " + str(best_paramater))

  #find best score and round to 2dp
  best_accuracy = round(grid_search.best_score_,2)

  ######Calculate the features ranked as best dependent on grid search result#######

  #This code works out the best features from the parameters given above
  estimator = RandomForestClassifier(random_state=0, n_estimators =  best_paramater['n_estimators'], bootstrap = best_paramater['bootstrap'])

  selector = RFECV(estimator, step=1, cv= number_of_cv_folds, scoring = scoring_type)
  selector = selector.fit(x_train, y_train)

  ranked_features = selector.ranking_
  print("[INFO] Rnk of all features: " + str(ranked_features))

  #create list to store important features
  most_important_features = []

  #aqrure the index of all values which have 1 as their ranking
  feat_index = np.where(ranked_features == 1)
  print("[INFO] Rank 1 features index: " + str(feat_index))
  #append the best featurees to the list
  for value in feat_index:
    most_important_features.append(x_train_df.columns[value])

  #Append results to 
  best_results.append([ Algorithm,
                       best_paramater['n_estimators'],                     
                      scoring_type,
                      best_accuracy,
                      best_paramater['max_features'],
                      
  ])

  best_results.append([
                      most_important_features])
           
  return best_results.append

def testing_combination_arrays(x_train_varible, max_varible):

  x_train_columns = x_train_varible.shape[1]
  x_train_columns = int(round((x_train_columns/2),0))
  print("[INFO] Number of max features: " + str(x_train_columns))
  feature_combinations = [] 
  varible_combinations = []

  a = 1
  for value in range(0,x_train_columns):
    feature_combinations.append(a) 
    a+=1 

  b = 1
  for value in range(0, max_varible):
    varible_combinations.append(b) 
    b+=1

  return feature_combinations, varible_combinations


def calculate_cross_validation_value(y_train):

  arr = y_train.to_numpy()
  bin_arr = np.bincount(arr)
  return bin_arr.min()

In [4]:
#@title Functions

def grid_search_RF_function(x_train, y_train, feature_combinations, varible_combinations, number_of_cv_folds, best_results, x_train_df):

  best_results=[]
  Algorithm = "Random Forest"
  forest_reg = RandomForestClassifier(random_state=0,bootstrap = False)

  #Set up parameter grid
  param_grid = [
      {'bootstrap': [False], 'n_estimators': varible_combinations, 'max_features': feature_combinations}
  ]

  #scoring_type can be used to choose what method to compare the models on
  scoring_type = 'f1'

  print("[INFO] Number of fold: " + str(number_of_cv_folds))
  
  grid_search = GridSearchCV(estimator= forest_reg, param_grid= param_grid,  cv= number_of_cv_folds , scoring=scoring_type,  return_train_score=True, n_jobs=-1)

  grid_search.fit(X= x_train, y= y_train)

  #Aquire best parameter for model
  best_paramater = grid_search.best_params_
  print("[INFO] Best parameter: " + str(best_paramater))

  #find best score and round to 4dp
  best_accuracy = round(grid_search.best_score_,4)
  print("[INFO] Score: " + str(best_accuracy))
  #Append results to 
  best_results.append([ Algorithm,
                        scoring_type,
                        best_accuracy,
                        best_paramater['n_estimators'],
                        best_paramater['max_features'],
                      
  ])


  return best_results, best_paramater

In [5]:
def get_best_model_input_data(x_train, y_train, number_of_cv_folds, best_paramater, best_results, x_train_df):

  model_check = []
  best_features =  []
  #create the best best model and fit with data to see inputs used
  RF = RandomForestClassifier(random_state=0, n_estimators = best_paramater['n_estimators'], bootstrap = best_paramater['bootstrap'], max_features = best_paramater['max_features'] )
  RF = RF.fit(x_train, y_train)

  bad_input = []

  #Aquire the festes that are important
  Feature_importance = RF.feature_importances_
  #copy dataframe so values can be removed
  updated_x_train = x_train_df.copy()

  a = 0
  #Every feauture which has 0 value is removed
  for value in Feature_importance:
    if(value == 0):
      bad_input.append(x_train_df.columns[a])
    a+=1

  print("[INFO] Number of removed inputs: " + str(len(bad_input)))

  for value in bad_input:
    updated_x_train = updated_x_train.drop([value], axis='columns')

  updated_x_train_df = updated_x_train.copy()

  #### Plot results of important features
  importance = np.abs(RF.feature_importances_)
  feature_names = np.array(x_train_df.columns)
  f, ax = plt.subplots(figsize=(30,5))
  plt.bar(height=importance, x=feature_names, )
  plt.title("Feature importances via feature_importances_ for Random Forest")
  plt.xticks(rotation='vertical')
  plt.show()


  best_features.append(updated_x_train_df.columns)

  scaler = MinMaxScaler()
  x_train = scaler.fit_transform(updated_x_train)
 # model = RandomForestClassifier(random_state=0, n_estimators = best_paramater['n_estimators'], bootstrap = best_paramater['bootstrap'], max_features = best_paramater['max_features'] )
  model = RandomForestClassifier(random_state=0, n_estimators = best_paramater['n_estimators'], bootstrap = best_paramater['bootstrap'], )

  y_train_pred = cross_val_predict(model, x_train, y_train, cv=number_of_cv_folds)

  conf = confusion_matrix(y_train, y_train_pred)
  print(conf)
  f1        = round(f1_score(y_train, y_train_pred), 4)
  precision = round(precision_score(y_train, y_train_pred), 4)
  recall    = round(recall_score(y_train, y_train_pred), 4)

  print("F1 score\tPurity\t\tRecovery")
  print(str(f1) + "\t\t" + str(precision) + "\t\t" + str(recall) )
    
  model_check.append([f1, precision, recall, conf])


  return best_features, model_check

In [6]:
header = ["Algorithm", "Scoring type", "Score", "Number of esitmators",  "Max features"]
model_check_header = ["f1", "Purity", "Recovery", "Confusion matrix"]
best_features_header = ["Imporant Features"]

In [ ]:
best_results = []
model_check = []
best_features = []
for key in uploaded:
    print("File uploaded: " +str(key))
    loaded_file = pd.read_csv(io.BytesIO(uploaded[key]))
    #loaded_file = loaded_file.abs()

    x_train_df = loaded_file.drop(['Label'], axis='columns')
    y_train = loaded_file.Label

    #make data scaled between 0 and 1
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train_df)

    #calcualte the largest possible cross validation
    number_of_cv_folds = calculate_cross_validation_value(y_train)
    print("[INFO] Number of CV folds: " + str(number_of_cv_folds))

    #Calualte the all feature combinations and create list for vaible combinations
    feature_combinations, varible_combinations = testing_combination_arrays(x_train, 20)
    print("[INFO] Varible combinations: " + str(varible_combinations))
    print("[INFO] Feature combinations: " + str(feature_combinations))

    #Perform grid search for Random Forest 
    best_results, best_paramater = grid_search_RF_function(x_train, y_train, feature_combinations, varible_combinations, number_of_cv_folds, best_results, x_train_df)
    best_features, model_check = get_best_model_input_data(x_train, y_train, number_of_cv_folds, best_paramater, best_results, x_train_df)


    with open(str(key) + '_random_forest' +  '.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(best_results)
        writer.writerow(model_check_header)
        writer.writerows(model_check)
        writer.writerow(best_features_header)
        writer.writerows(best_features)

    files.download(str(key) + '_random_forest' + '.csv') 


File uploaded: SSteel_25-75WF_dataset.csv
[INFO] Number of CV folds: 14
[INFO] Number of max features: 34
[INFO] Varible combinations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[INFO] Feature combinations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
[INFO] Number of fold: 14
